In [ ]:
##내가 썼던 코드
# 
# 



from peft import get_peft_config, get_peft_model, PeftType, PromptEncoder, PromptEncoderConfig
from datasets import load_dataset
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

import makeDataset
import os
import json
import torch
from transformers import (
    AutoProcessor,
    EncodecModel,
    TrainingArguments,
    Trainer,
    AutoTokenizer,
    MusicgenForConditionalGeneration,
    DataCollatorForLanguageModeling
)
from peft import PromptEncoderConfig, get_peft_config, get_peft_model, TaskType, PeftType, PromptEncoder, PromptEncoderConfig
from datasets import Dataset
import librosa

# warnings.filterwarnings("ignore", category=FutureWarning)
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")


# T5
text_encoder = model.text_encoder 
# 데이터셋 로드
dataset = load_dataset("makeDataset.py", trust_remote_code=True)



# # 전역 설정값 지정
task = "mrpc"
num_epochs = 3
lr = 1e-3
batch_size = 2


# P-tuning 설정
peft_config = PromptEncoderConfig(
    peft_type="P_TUNING",
    task_type=TaskType.SEQ_2_SEQ_LM, # 연속적인 데이터(음악, 텍스트)임을 명시
    num_virtual_tokens=20,
    token_dim=text_encoder.config.hidden_size,
    num_transformer_submodules=1,
    num_attention_heads=text_encoder.config.num_heads,
    num_layers=text_encoder.config.num_layers,
    encoder_reparameterization_type="MLP",
    encoder_hidden_size=text_encoder.config.d_ff,
)
text_encoder = get_peft_model(text_encoder, peft_config)

model.text_encoder = text_encoder



###
#모델 파라미터 freeze (피튜닝만 진행)
for param in model.parameters():
    param.requires_grad = False

for param in model.text_encoder.parameters():
    param.requires_grad = False

for param in model.text_encoder.prompt_encoder.parameters():
    param.requires_grad = True



#########

parameter_size = model.text_encoder.print_trainable_parameters()

audio_encoder = EncodecModel.from_pretrained("facebook/encodec_32khz").eval()

data_dir = './dataset_wav'

data = {'text': [], 'audio': []}



##데이터 준비 
for file_name in os.listdir(data_dir):
    if file_name.endswith('.json'):
        json_path = os.path.join(data_dir, file_name)
        with open(json_path, 'r') as f:
            json_data = json.load(f)
        description = json_data['description']
        base_name = os.path.splitext(file_name)[0]
        mp3_file = base_name + '.wav'
        mp3_path = os.path.join(data_dir, mp3_file)
        if os.path.exists(mp3_path):
            data['text'].append(description)
            data['audio'].append(mp3_path)
            
        else:
            print(f"Warning: mp3 file for {file_name} not found.")

dataset = Dataset.from_dict(data)

max_audio_length = 320000  # 10초 * 32,000Hz

def preprocess_function(example):
    audio_file = example['audio']
    try:
        audio_array, sr = librosa.load(audio_file, sr=32000)
    except Exception as e:
        print(f"Error loading {audio_file}: {e}")
        return None

    audio_tensor = torch.tensor(audio_array).unsqueeze(0).unsqueeze(1)  # Shape: (1, 1, sequence_length)

    if audio_tensor.shape[-1] > max_audio_length:
        audio_tensor = audio_tensor[..., :max_audio_length]
    elif audio_tensor.shape[-1] < max_audio_length:
        pad_size = max_audio_length - audio_tensor.shape[-1]
        audio_tensor = torch.nn.functional.pad(audio_tensor, (0, pad_size))

    inputs = processor(
        text=[example['text']],
        audio=audio_tensor,  # 오디오 텐서 추가
        padding=True,
        return_tensors="pt",
    )

    with torch.no_grad():
        encoded_audio = audio_encoder.encode(audio_tensor)

    audio_codes = encoded_audio.audio_codes  # shape: (frames, batch_size, num_codebooks, seq_len)
    frames, batch_size, num_codebooks, seq_len = audio_codes.shape
    labels = audio_codes.reshape(batch_size * num_codebooks, seq_len)

    return {
        'input_ids': inputs['input_ids'][0],
        'attention_mask': inputs['attention_mask'][0],
        'labels': labels
    }

# 데이터 수집기 설정
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # 마스크 언어 모델링 비활성화
)



# training 인자 설정
training_args = TrainingArguments(
    output_dir='./musicgen_ptuning_output',
    per_device_train_batch_size=1, #학습 배치 크기
    num_train_epochs=3, #epoch
    logging_steps=10, 
    save_steps=10,
    learning_rate=1e-4, #학습률
    remove_unused_columns=True,
)

# Trainer 인스턴스 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

# 모델 학습
trainer.train()

# 튜닝된 모델 저장
model.save_pretrained("musicgen-ptuning-tuned")

c:\Users\a\Desktop\MusicGen\.conda\Lib\site-packages\transformers\models\encodec\modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_

TypeError: 'NoneType' object is not callable

In [6]:
##20241120 조교님이 주신 코드
# 
# 

import os
import json
import torch
from transformers import (
    AutoProcessor,
    EncodecModel,
    TrainingArguments,
    Trainer,
    MusicgenForConditionalGeneration
)
from peft import PromptEncoderConfig, get_peft_model, TaskType
from datasets import Dataset
import librosa

# warnings.filterwarnings("ignore", category=FutureWarning)
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

# T5
text_encoder = model.text_encoder

peft_config = PromptEncoderConfig(
    peft_type="P_TUNING",
    task_type=TaskType.SEQ_2_SEQ_LM,
    num_virtual_tokens=20,
    token_dim=text_encoder.config.hidden_size,
    num_transformer_submodules=1,
    num_attention_heads=text_encoder.config.num_heads,
    num_layers=text_encoder.config.num_layers,
    encoder_reparameterization_type="MLP",
    encoder_hidden_size=text_encoder.config.d_ff,
)

text_encoder = get_peft_model(text_encoder, peft_config)

model.text_encoder = text_encoder

for param in model.parameters():
    param.requires_grad = False

for param in model.text_encoder.parameters():
    param.requires_grad = False

for param in model.text_encoder.prompt_encoder.parameters():
    param.requires_grad = True

parameter_size = model.text_encoder.print_trainable_parameters()

audio_encoder = EncodecModel.from_pretrained("facebook/encodec_32khz").eval()

data_dir = './dataset_wav'

data = {'text': [], 'audio': []}


for file_name in os.listdir(data_dir):
    if file_name.endswith('.json'):
        json_path = os.path.join(data_dir, file_name)
        with open(json_path, 'r') as f:
            json_data = json.load(f)
        description = json_data['description']
        base_name = os.path.splitext(file_name)[0]
        mp3_file = base_name + '.wav'
        mp3_path = os.path.join(data_dir, mp3_file)
        if os.path.exists(mp3_path):
            data['text'].append(description)
            data['audio'].append(mp3_path)
            
        else:
            print(f"Warning: mp3 file for {file_name} not found.")

dataset = Dataset.from_dict(data)

max_audio_length = 320000  # 10초 * 32,000Hz

def preprocess_function(example):
    audio_file = example['audio']
    try:
        audio_array, sr = librosa.load(audio_file, sr=32000)
    except Exception as e:
        print(f"Error loading {audio_file}: {e}")
        return None

    audio_tensor = torch.tensor(audio_array).unsqueeze(0).unsqueeze(1)  # Shape: (1, 1, sequence_length)


    if audio_tensor.shape[-1] > max_audio_length:
        audio_tensor = audio_tensor[..., :max_audio_length]
    elif audio_tensor.shape[-1] < max_audio_length:
        pad_size = max_audio_length - audio_tensor.shape[-1]
        audio_tensor = torch.nn.functional.pad(audio_tensor, (0, pad_size))

    inputs = processor(
        text=[example['text']],
        padding=True,
        return_tensors="pt",
    )

    with torch.no_grad():
        encoded_audio = audio_encoder.encode(audio_tensor)

    audio_codes = encoded_audio.audio_codes  # shape: (frames, batch_size, num_codebooks, seq_len)
    frames, batch_size, num_codebooks, seq_len = audio_codes.shape
    labels = audio_codes.reshape(batch_size * num_codebooks, seq_len)

    return {
        'input_ids': inputs['input_ids'][0],
        'attention_mask': inputs['attention_mask'][0],
        'labels': labels
    }


processed_dataset = dataset.map(preprocess_function, batched=False)
processed_dataset = processed_dataset.filter(lambda x: x is not None)

processed_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

def data_collator(features):
    if len(features) == 0:
        return None

    return {
        'input_ids': torch.stack([f['input_ids'] for f in features]),
        'attention_mask': torch.stack([f['attention_mask'] for f in features]),
        'labels': torch.stack([f['labels'] for f in features]),
    }

# Training Arguments
training_args = TrainingArguments(
    output_dir='./musicgen_ptuning_results',
    per_device_train_batch_size=1,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=10,
    learning_rate=1e-4,
    remove_unused_columns=True,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    data_collator=data_collator, 
    # tokenizer=processor.tokenizer,  # tokenizer
)
trainer.train()

Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_be

trainable params: 14,178,048 || all params: 123,806,592 || trainable%: 11.4518


Parameter 'function'=<function preprocess_function at 0x0000021492779620> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map:   0%|          | 0/10 [00:04<?, ? examples/s]


KeyboardInterrupt: 